In [1]:
# loading libraries
import psycopg2
import pandas as pd
import numpy as np
import pandas.io.sql as psql

In [2]:
# connecting to redshift 
dbname1, host1, port1, user1, password1 = pd.read_csv("credentials.csv")
con = psycopg2.connect(dbname= dbname1, host= host1, port= port1, user= user1, password= password1)

In [3]:
# get a cursor from DB connection
cur = con.cursor()

In [4]:
# execute query and load into pandas df
data = psql.read_sql('SELECT * FROM "clipper"."rpt-cashpass3";', con)

In [5]:
#data.sort_values(['applicationserialnumber','year', 'month'])

In [6]:
len(data)

14854169

In [7]:
#drop if no muni use (cash or pass) in the month
data.drop(data[(data.countpass == 0) & (data.countcash == 0)].index, inplace=True)

In [8]:
len(data)

10476386

In [9]:
#add total ride column
data = data.assign(totrides= data.countpass + data.countcash)

In [10]:
#add rides percentages
data = data.assign(passpercent= data.countpass / data.totrides)
data = data.assign(cashpercent= data.countcash / data.totrides)

In [11]:
#drop all records for clipper card if took less than 10 rides in a month
data2 = data.groupby('applicationserialnumber').filter(lambda x: x['totrides'].min()>10 )
data2.reset_index(drop=True, inplace=True) # reset index

In [ ]:
len(data2)

1505712

These histograms look at the average rides per month for Muni users, not controlling for those who have switched between pass and cash. We want to control for just "switchers" because there are potentially confounding factors, ie a certain user type tends to use a pass and also tends to take a lot of rides. We want to keep all other factors constant. 

In [ ]:
%matplotlib inline
data2.countpass.plot.hist()

In [ ]:
data2.countcash.plot.hist()

In [ ]:
#find the total rides per user across the data
data2 = data2.assign(monthcount= 1) #add comlumn to count number of months 
data3 = data2.groupby('applicationserialnumber')[["countpass","countcash","totrides","monthcount"]].sum()
data3.reset_index(inplace=True) # reset index

In [ ]:
#drop people who definitely didnt switch / wouldn't be helpful for this analysis
# bc they haven't taken more than 50 rides in either category acorss the two years
switchers = data3.drop(data3[(data3.countpass < 50) | (data3.countcash < 50)].index)

In [ ]:
len(switchers)

In [ ]:
#drop people who only have cliper card data for less than 12 months
switchers2 = switchers.drop(switchers[(switchers.monthcount < 12)].index)

In [ ]:
len(switchers2)

In [ ]:
#filter monthly transaction data for only the identified switchers
switchersdata = data2[data2['applicationserialnumber'].isin(list(switchers2.applicationserialnumber))]

In [ ]:
#replace zero with NaN so the the mean doesn't include those months
switchersdata = switchersdata.replace(0, np.NaN)

In [ ]:
#switchersdata.sort_values(['applicationserialnumber','year', 'month'])

In [ ]:
#find the average rides per user per month
avgswitchers = switchersdata.groupby('applicationserialnumber')[["countpass","countcash","totrides","monthcount"]].mean()
avgswitchers.reset_index(inplace=True) # reset index

In [ ]:
#find the total rides per user per month
avgswitchers = (switchersdata.groupby('applicationserialnumber')
.agg({'countpass':['mean','sum'], 'countcash':['mean','sum'],'totrides':['mean','sum'],'monthcount':'sum'})
.reset_index()
)

In [ ]:
#rename columns
avgswitchers.columns = ["_".join(x) for x in avgswitchers.columns.ravel()]

In [ ]:
#rename columns
avgswitchers = avgswitchers.rename(columns={'applicationserialnumber_':'applicationserialnumber', 'monthcount_sum':'totalmonths',
                             'countcash_mean':'avgcashpermonth', 'countcash_sum' : 'totalcashrides', 
                            'countpass_mean':'avgpasspermonth', 'countpass_sum' : 'totalpassrides',
                            'totrides_mean':'avgridespermonth', 'totrides_sum' : 'totalrides'})

In [ ]:
#drop outlier
avgswitchers = avgswitchers.drop(avgswitchers[(avgswitchers.avgpasspermonth > 300)].index)

In [ ]:
#add percentage of avg pass and cash per month
avgswitchers = avgswitchers.assign(avgpassridesperc= avgswitchers.totalpassrides / avgswitchers.totalrides)

avgswitchers = avgswitchers.assign(avgcashridesperc= avgswitchers.totalcashrides / avgswitchers.totalrides)

This scatter plot shows the average number of rides taken using eCash and pass per month by those who have switched. Each dot represents a user. The scale on the pass axis is much larger suggesting users who have used both pass and eCash take more rides when they have a pass. 

In [ ]:
avgswitchers.plot.scatter(x = 'avgcashpermonth', y = 'avgpasspermonth', alpha = 0.8)

This histogram shows the total number of average rides per month switchers took when they used a pass.

In [ ]:
avgswitchers.avgpasspermonth.plot.hist().axvline(avgswitchers.avgpasspermonth.mean(), color='b', linestyle='dashed', linewidth=2)


In [ ]:
#what is the average number of rides per a month that switchers took when they had a pass
avgswitchers.avgpasspermonth.mean()

This histogram shows the total number of average rides per month switchers took when they used a cash.

In [ ]:
avgswitchers.avgcashpermonth.plot.hist().axvline(avgswitchers.avgcashpermonth.mean(), color='b', linestyle='dashed', linewidth=2)

In [ ]:
#what is the average number of rides per a month that switchers took when they used eCash
avgswitchers.avgcashpermonth.mean()